In [9]:
import pandas as pd
import numpy as np

data_discover = pd.read_csv(r'Discover_Часовые_общий_актуальные_данные.csv', sep=';', na_values='N', decimal=',')

names = ['bx_gsm', 'by_gsm', 'bz_gsm', 'bt', 'proton_density', 'proton_speed', 'proton_temperature']

In [11]:
data_discover.iloc[range(1256,1268), [4, 9, 10, 11, 20, 21]]

,bt,bx_gsm,by_gsm,bz_gsm,proton_density,proton_temperature
1256,5.807213,2.268852,-4.409508,2.625738,15.559672,218285.24590
1257,7.372292,-1.107917,-6.313542,2.603542,17.140000,94458.79167
1258,NaN,NaN,NaN,NaN,NaN,NaN
1259,NaN,NaN,NaN,NaN,NaN,NaN
1260,NaN,NaN,NaN,NaN,NaN,NaN
1261,NaN,NaN,NaN,NaN,NaN,NaN
1262,6.326842,3.803684,3.412105,3.584211,NaN,NaN
1263,5.303443,3.611967,-2.949180,2.031148,12.658125,210951.03130
1264,5.082787,3.703115,-1.958852,2.768197,13.029016,199954.37700
1265,4.494262,3.891475,-1.589672,1.217869,12.690656,199802.55740


In [14]:
pd.set_option('future.no_silent_downcasting', True)

for elem in names:
    counter, out = -1, 0
    for index, value in data_discover[elem].items():
        if (value == 'N') & (counter < 12):
            counter += 1
        elif (value == 'N') & (counter >= 12):
            counter = 0
            out = 1
        elif (value != 'N') & (counter in range(1, 13)) & (out == 0):
            data_discover.loc[range(index-counter, index+1), elem] = np.NaN
            data_slice = data_discover.loc[range(index-counter-1, index+2), elem]
            # print(data_slice)
            data_slice_obj = data_slice.infer_objects(copy=False)
            data_discover.loc[range(index-counter-1, index+2), elem] = data_slice_obj.interpolate(method='linear')
            counter = 0
        else:
            counter = 0
            out = 0


data_discover.to_csv("Discover_с_интреполяцией12_актуальные_данные_до_22.09.2024.csv", sep=';', index=False, encoding='utf-8')

In [15]:
data_discover.iloc[range(1256,1269), [4, 9, 10, 11, 20, 21]]

,bt,bx_gsm,by_gsm,bz_gsm,proton_density,proton_temperature
1256,5.807213,2.268852,-4.409508,2.625738,15.559672,218285.24590
1257,7.372292,-1.107917,-6.313542,2.603542,17.140000,94458.79167
1258,NaN,NaN,NaN,NaN,NaN,NaN
1259,NaN,NaN,NaN,NaN,NaN,NaN
1260,NaN,NaN,NaN,NaN,NaN,NaN
1261,NaN,NaN,NaN,NaN,NaN,NaN
1262,6.326842,3.803684,3.412105,3.584211,NaN,NaN
1263,5.303443,3.611967,-2.949180,2.031148,12.658125,210951.03130
1264,5.082787,3.703115,-1.958852,2.768197,13.029016,199954.37700
1265,4.494262,3.891475,-1.589672,1.217869,12.690656,199802.55740
